Import libraries.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from pandas_profiling import ProfileReport

Read data in.

In [ ]:
census_df = pd.read_csv("CensusN4.csv") # Read CSV in as dataframe

In [ ]:
census_df.head() # Snippet from top of the dataset

In [ ]:
census_df.tail() # Snippet from bottom of dataset

In [ ]:
census_df.shape # Get row and column count

In [ ]:
profile = ProfileReport(census_df, title = "Census Data Report", explorative = True) # Produce profile report
profile.to_file("census_report.html") # Save as HTML file

## Cleaning

### Missing Entries

Replace blank values and 'Na' with Numpy's nan value.

In [ ]:
census_df.replace([' '], [np.nan], inplace=True)
census_df.replace(['Na'], [np.nan], inplace=True)

See how many null values exist in each column.

In [ ]:
census_df.isna().sum()

I'll work my way down, beginning with house number and street.

In [ ]:
census_df[census_df['House Number'].isna()] # Return rows with missing house number

Let's check if anyone with her surname lives down the same street as her.

In [ ]:
census_df.loc[census_df['Surname'] == census_df.iloc[59]['Surname']] # Return rows with same surname

We can deduce that Leigh Hobbs-Jones is Ann's Husband and lives with her at 23.

In [ ]:
census_df.at[59, 'House Number'] = 23 # Set to 23

Street column is next, with 2 missing values.

In [ ]:
census_df[census_df['Street'].isna()] # Return rows with missing streets

Let's check if anyone with the surname Carter lives in the same house number.

In [ ]:
census_df.loc[census_df['Surname'] == census_df.iloc[508]['Surname']] # Return rows with same surname

The only people living at a number one house with the surname Carter are Kerry and Paul, but Kerry is the head and there can't be two heads in one house. Furthermore, there is noone above him or below him in the census who he may live with.

In [ ]:
census_df[506:511] # Return rows surrounding subject

However, it's clear the census is grouped into streets, and we can see that there is a number 1 missing for Westhatch Street. While it's possible that he lives in his own street which has been omitted, it's highly unlikely.

In [ ]:
census_df.at[508, 'Street'] = 'Westhatch Street' # Set to Westhatch Street

Let's perform the same check for Gillian Thompson.

In [ ]:
census_df.loc[census_df['Surname'] == census_df.iloc[5715]['Surname']] # Return rows with same surname

We know that Gillian is married and Intramystical, so we have a good chance of finding her partner, and then her street.

The only possible candidate is Clive Thompson. Perhaps most obviously is that he is directly below Gillian in the census, but this is also proven by the following - he lives at the same house number, he is Intramystical, he's husband to the head of the house (Gillian), and he's a similar age.

In [ ]:
census_df.at[5715, 'Street'] = "Dickinson Avenue" # Set to Dickinson Ave

With addresses completed, I'm going to combine house number and street to create a single address for each user, for ease of use.

In [ ]:
census_df['Address'] = census_df['House Number'].astype(str) + ", " + census_df['Street'] # Concatenate house number and street

After closer inspection of the age column, some are written as strings while others are floats.

In [ ]:
census_df['Age'].value_counts(dropna=False) # Return value counts

Convert to integers.

In [ ]:
census_df['Age'] = census_df['Age'].replace(['One', 'Nine', 'Ten'], ['1', '9', '10']) # Replace words with integers

In [ ]:
census_df['Age'] = census_df['Age'].astype(float).astype(int) # Convert to floats

Get minimum and maximum ages.

In [ ]:
census_df['Age'].min() # Minimum

In [ ]:
census_df['Age'].max() # Maximum

Next up, names.

In [ ]:
census_df[census_df['Surname'].isna()] # Return rows with misisng surname

Many of them have two names in the first name column, presumably by accident, so we can manipulate the string to split this into a first and a surname.

In [ ]:
census_df.loc[census_df['First Name'].str.split().str.len() == 2, 'Surname'] = census_df['First Name'].str.split().str[-1] # Set surname equal to second half of string
census_df.loc[census_df['First Name'].str.split().str.len() == 2, 'First Name'] = census_df['First Name'].str.split().str[0] # Set first name equal only to first half

In [ ]:
census_df[census_df['Surname'].isna()] # Return remaining rows with missing surnames

For these two individuals, we can try deduce their surnames from their families.

In [ ]:
census_df.loc[census_df['Address'] == census_df.iloc[2588]['Address']] # Return rows with same address

Their surname could be either Fletcher or Walters.

In [ ]:
# For now, I'll set it to Fletcher as it's the more frequent in the family
census_df.at[2588, 'Surname'] = "Fletcher"

In [ ]:
census_df.loc[census_df['Address'] == census_df.iloc[4998]['Address']] # Return rows with same address

We can safely assume Jill's surname is Blackburn.

In [ ]:
census_df.at[4998, 'Surname'] = "Blackburn" # Set to Blackburn

Tackle marital status column next.

Marital status is a categorical column, so we will take a look at the possible values.

In [ ]:
census_df['Marital Status'].value_counts(dropna=False) # Return value counts

Obviously, 'Div-orced' and "Don't Ask!" are invalid so we will deal with them.

For 'Div-orced', we will simply replace it with 'Divorced'.

In [ ]:
census_df['Marital Status'].replace('Div-orced', 'Divorced', inplace=True) # Replace spelling mistake

Let's take a look at anyone who answered "Don't Ask!".

In [ ]:
census_df.loc[census_df['Marital Status'] == "Don't Ask!"] # Inspect anomaly

Inspect their household more closely.

In [ ]:
census_df.loc[census_df['Address'] == census_df.iloc[5924]['Address']] # Return rows with their address

They live alone, so they're either divorced, single, or widowed.

Drop row for now,it's hard to know.

In [ ]:
census_df.drop(5924, inplace = True) # Drop row from data
census_df.reset_index(drop=True, inplace=True) # Reset index

A number of rows have a null value value for Marital Status so I'll take a closer look.

In [ ]:
census_df[census_df['Marital Status'].isna()] # Return rows with null marital status

It's evident that all of them are minors so we can assume they're single.

In [ ]:
census_df.loc[census_df['Age'] < 18, 'Marital Status'] = "Minor" # Set all minors marital status to reflect their age

Next, I'll address gender, which is categorical.

In [ ]:
census_df['Gender'].unique() # Return unique entries

Replace with female.

In [ ]:
census_df['Gender'].replace('Fe-male', 'Female', inplace=True) # Replace spelling mistake

In [ ]:
census_df[['First Name', 'Surname', 'Age', 'Relationship to Head of House', 'Gender']][census_df['Gender'].isna()] # Return rows with missing gender

Our first subject, Elizabeth Murray can be presumed to be female from her name alone.

Grace Thomas and Shirley Johnston also have female names, and their gender is further evidenced by their relationship to head of house being 'Daughter'.

In [ ]:
census_df['Gender'].fillna("Female", inplace=True) # Fill missing values with female

Next is the occupation column.

In [ ]:
census_df[['First Name', 'Surname', 'Age', 'Occupation']][census_df['Occupation'].isna()] # Return rows with missing occupation

Like every other child her age, we can assume Paige Nolan to be a student.

In [ ]:
census_df.at[4195, 'Occupation'] = "Student" # Set to student

Let's look at Leanne's household for a better idea.

In [ ]:
census_df.loc[census_df['Address'] == census_df.iloc[6419]['Address']] # Inspect by address

She's a single mother so presumably, she will have a job doing something.

There's no way of accurately guessing what her occupation is, so she can be removed instead.

In [ ]:
census_df.drop(6419, inplace = True) # Drop from data
census_df.reset_index(drop=True, inplace=True) # Reset index

Infirmity.

In [ ]:
census_df['Infirmity'].unique() # Return unique entries

Infirmity percentages.

In [ ]:
census_df['Infirmity'].value_counts(dropna=False) # Return value counts

Replace fake infirmities.

In [ ]:
# Replace fake infirmities with None
census_df['Infirmity'].replace('Silly', 'None', inplace=True)
census_df['Infirmity'].replace('Toothache', 'None', inplace=True)
census_df['Infirmity'].replace('Skygazer', 'None', inplace=True)

In [ ]:
census_df[['Age', 'Gender', 'Occupation', 'Infirmity']][census_df['Infirmity'].isna()] # Return rows with missing infirmity field

There's no telling signs of infirmities we can gather, so we will just presume that the people above have no infirmity, as the vast majority of the population don't.

In [ ]:
census_df['Infirmity'].fillna('None', inplace=True) # Fill missing values with None

Religion.

In [ ]:
census_df['Religion'].unique() # Return unique entries

Potentially, we could write a function which examines each person and assesses their household's religion.

Here are the counts of each religion.

In [ ]:
census_df['Religion'].value_counts(dropna=False) # Return value counts

In [ ]:
census_df[census_df['Religion'].isna()] # Return rows with missing religion

Most of missing values belong to minors, set to undeclared.

In [ ]:
census_df.loc[census_df['Age'] < 18, 'Religion'] = "Undeclared" # Set minors to undeclared

In [ ]:
census_df['Religion'].value_counts(dropna=False) # Return new value counts

Get rid of fake religions.

In [ ]:
# Replace fake religions with None
fake_religions = ["Washer", "Angry", "Wisher", "Nopenope"]
for rel in fake_religions:
    census_df['Religion'].replace(rel, 'None', inplace=True)

Assume that those with NaN have no religion.

In [ ]:
#census_df['Religion'].fillna('None', inplace=True) # Fill missing values with None

Or, use frequency.

In [ ]:
s = census_df['Religion'].value_counts(normalize=True)

census_df.loc[census_df['Religion'].isna(), 'Religion'] = np.random.choice(s.index, p=s.values, size=census_df['Religion'].isna().sum())

Which gives us the new value counts:

In [ ]:
census_df['Religion'].value_counts(dropna=False)

All missing values should now be dealt with.

In [ ]:
census_df.isna().sum() # Check for remaining missing values

### Other Features

Let's check the values of other categorical features, to ensure that they are valid.

In [ ]:
census_df['Relationship to Head of House'].unique() # Return unique entries

We can assume 'None' to mean that the person in question is a Visitor of the house.

In [ ]:
census_df['Relationship to Head of House'].replace('None', 'Visitor', inplace=True) # Replace none with visitor

Niece is spelt incorrectly so we can simply replace it.

In [ ]:
census_df['Relationship to Head of House'].replace('Neice', 'Niece', inplace=True) # Replace spelling mistake

Next up is occupation.

In [ ]:
census_df['Occupation'].nunique() # Get unique count

There's 1040 different occupations so it's impossible to check each occupation is valid.

## Analysis

Let's begin further analysis, with an examination of age distribution.

In [ ]:
np.std(census_df['Age']) # Standard deviation

In [ ]:
# Histogram depicting age
hist = sns.displot(data = census_df, x = "Age", binwidth=10)
hist.set(title = "Age of Census") # Set title

In [ ]:
# Another histogram, but with gender taken into consideration
hist = sns.displot(data = census_df, x = "Age", binwidth=20, hue = "Gender", multiple="dodge")
hist.set(title = "Age of Census, per Gender") # Set title

Population Pyramid

In [ ]:
# Put ages into new format
age_df = pd.DataFrame({'Age': ['0-4','5-9','10-14','15-19','20-24','25-29','30-34','35-39','40-44','45-49','50-54','55-59','60-64','65-69','70-74','75-79','80-84','85-89','90+'],
                          'Male': [-185, -211, -237, -275, -224, -189, -215, -254, -265, -201, -209, -186, -144, -123, -97, -72, -50, -24, -16],
                          'Female': [141, 208, 218, 266, 263, 218, 293, 288, 329, 269, 241, 217, 163, 140, 120, 78, 51, 16, 16]})

In [ ]:
age_order = ['90+','85-89','80-84','75-79','70-74','65-69','60-64','55-59','50-54','45-49','40-44','35-39','30-34','25-29','20-24','15-19','10-14','5-9','0-4'] # Order as such
plt.figure(figsize=(10, 13)) # Set figure size
bar_plot = sns.barplot(x='Male', y='Age', data = age_df, order=age_order) # Plot males
bar_plot = sns.barplot(x='Female', y='Age', data=age_df, order=age_order) # Plot females
bar_plot.set(xlabel = "Population", ylabel = "Age", title="Population Pyramid") # Set labels and title

In [ ]:
# Show descriptive stats of age in box plot
age_box = sns.boxplot(data = census_df, x = "Age")
age_box.set(title="Boxplot of Age")

Now, I'm going to look at unemployment trends.

In [ ]:
census_df['Occupation'].value_counts() # Return value counts for occupation

Count number of children/students.

In [ ]:
student_count = len(census_df[census_df['Occupation'] == 'Student']) + len(census_df[census_df['Occupation'] == 'University Student']) + len(census_df[census_df['Occupation'] == 'Child']) + len(census_df[census_df['Occupation'] == 'PhD Student']) # Count students and children
student_count

Count number of retirees.

In [ ]:
retirees = len(census_df[census_df['Occupation'].str.contains("Retired")]) # Count retired
retirees

Subtract to calculate number of economically active people.

In [ ]:
econ_active = len(census_df) - retirees - student_count # Subtract to get economically active population
econ_active

Divide unemployed count by number of economically active, and multiply by 100.

In [ ]:
# Calculate unemployment rate
unemployment_rate = len(census_df[census_df['Occupation'] == 'Unemployed']) / econ_active
unemployment_rate * 100

Plot ages of unemployed people.

In [ ]:
# Plot age histogram of unemployed people
hist_plot = sns.displot(data = census_df[census_df['Occupation'] == 'Unemployed'], x = "Age", binwidth=10)
hist_plot.set(title = "Age of Unemployed People")

Plot genders of unemployed people.

In [ ]:
# Plot genders of unemployed people
bar_plot = sns.displot(data = census_df[census_df['Occupation'] == 'Unemployed'], x = "Gender", multiple="dodge", binwidth=10)
bar_plot.set(title = "Gender of Unemployed People")

Count total gender gap in population.

In [ ]:
census_df['Gender'].value_counts() # Return gender value counts for comparison

Work out percentages of unemployed men and women.

In [ ]:
male_unemployed = len(census_df[(census_df['Occupation']== 'Unemployed') & (census_df['Gender']=='Male')])
female_unemployed = len(census_df[(census_df['Occupation']== 'Unemployed') & (census_df['Gender']=='Female')])

In [ ]:
male_unemployed/len(census_df[census_df['Occupation'] == 'Unemployed']) * 100

In [ ]:
female_unemployed/len(census_df[census_df['Occupation'] == 'Unemployed']) * 100

Religion next.

In [ ]:
census_df['Religion'].value_counts(dropna=False) # Value counts for religion

Plot religion bar chart.

In [ ]:
# Plot counts of each religion
bar_plot = sns.displot(data = census_df, x = "Religion", binwidth=10)
bar_plot.set(title="Religion of Census")
bar_plot.set_xticklabels(rotation=60)

Plot categorical plot of religions vs age.

In [ ]:
cat_plot = sns.catplot(data = census_df, x = "Religion", y = "Age")
cat_plot.set_xticklabels(rotation=60)
cat_plot.set(title="Religions vs. Age")

Boxplot of religions vs age.

In [ ]:
box_plot = sns.boxplot(data = census_df, x = "Religion", y = "Age")
box_plot.set_xticklabels(box_plot.get_xticklabels(),rotation=30)
box_plot.set(title="Religions vs. Age")

Occupancy.

In [ ]:
census_df['Address'].nunique() # No. of unique addresses

In [ ]:
census_df['Address'].count() # Number of people

In [ ]:
census_df['Address'].count() / 2454 # Average per household

Students.

In [ ]:
census_df['Occupation'][census_df['Occupation'] == 'University Student'].count() # Counts number of students

In [ ]:
409 / census_df['Occupation'].count() * 100 # Percentage of population who are students

Birth Rate

In [ ]:
babies = census_df['Age'][census_df['Age'] == 0].count() # Count number of babies
babies

In [ ]:
birth_rate =  (babies / len(census_df)) * 100000 # Calculate birth rate

In [ ]:
print("Current rate:","%.2f" % birth_rate,"births per 100,000")

In [ ]:
four_yrs = census_df['Age'][census_df['Age'] == 4].count() # Count number of 4 year olds
four_yrs

In [ ]:
birth_rate_2 = (four_yrs / len(census_df)) * 100000 # Calculate birth rate of 5 years ago

In [ ]:
print("Past rate:","%.2f" % birth_rate_2,"births per 100,000")

Death Rate

In [ ]:
middle_age = len(census_df[(census_df['Age'] > 55) & (census_df['Age'] < 61)]) # Count number of 56-60
middle_age

In [ ]:
older = len(census_df[(census_df['Age'] > 60) & (census_df['Age'] < 66)]) # Count number of 61-65
older

In [ ]:
death_rate = middle_age - older # Subtract to get a death rate per 5 years

In [ ]:
print("Death rate of first group:",death_rate/5,"per annum") # Divide by 5

In [ ]:
old_age = len(census_df[(census_df['Age'] > 65) & (census_df['Age'] < 71)]) # Count number of 66-70
old_age

In [ ]:
older_2 = len(census_df[(census_df['Age'] > 70) & (census_df['Age'] < 76)]) # Count number of 71-75
older_2

In [ ]:
death_rate_2 = old_age - older_2 # Subtract to get a death rate per 5 years

In [ ]:
print("Death rate of second group:",death_rate_2/5,"per annum") # Divide by 5

In [ ]:
# Count number of 76-80 and 81-85
age_3 = len(census_df[(census_df['Age'] > 75) & (census_df['Age'] < 81)])
older_3 = len(census_df[(census_df['Age'] > 80) & (census_df['Age'] < 86)])

In [ ]:
death_rate_3 = age_3 - older_3 # Subtract for death rate

In [ ]:
print("Death rate of third group:",death_rate_3/5,"per annum") # Divide by 5

In [ ]:
# Repeat
age_4 = len(census_df[(census_df['Age'] > 85) & (census_df['Age'] < 91)])
older_4 = len(census_df[(census_df['Age'] > 90) & (census_df['Age'] < 96)])
death_rate_4 = age_4-older_4

In [ ]:
print("Death rate of fourth group:",death_rate_4/5,"per annum")

In [ ]:
#Repeat
age_5 = len(census_df[(census_df['Age'] > 95) & (census_df['Age'] < 101)])
older_5 = len(census_df[census_df['Age'] > 100])
death_rate_5 = age_5 - older_5

In [ ]:
print("Death rate of fifth group:",death_rate_5/5,"per annum")

In [ ]:
# Add deaths together for total
total_deaths = death_rate+death_rate_2+death_rate_3+death_rate_4+death_rate_5 

In [ ]:
total_death_rate = (total_deaths / len(census_df)) * 100000 # Per 100,000 people

In [ ]:
print("Total death rate:","%.2f" % total_death_rate,"per 100,000")

In [ ]:
2324/5 # Divide by 5 to get per annum